## AGAME Workflow. Get flux data

In [5]:
import os
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from icoscp.station import station
from icoscp.dobj import Dobj
import geopandas as gpd
from shapely.geometry import Point
import folium
import requests

# from icoscp_core.icos import auth
# auth.init_config_file()

In [6]:
def read_flux_df(file_path):
    print('\nReading:', file_path)

    df = pd.read_csv(file_path, na_values=-9999)
    original_num_columns = df.shape[1]
    cols_to_drop = df.columns[df.isna().all()].tolist()
    num_cols_dropped = len(cols_to_drop)

    print(f"Original number of columns: {original_num_columns}")
    print(f"Number of columns dropped: {num_cols_dropped}")
    print(f"Columns dropped: {cols_to_drop}\n")

    df_cleaned = df.drop(columns=cols_to_drop)

    return df_cleaned

def filter_era_columns(sites_table, df, list_col, nan_files):
    counts = Counter(list_col)
    list_col_in_all = set([item for item in list_col if counts[item] == (len(sites_table)-len(nan_files))])
    print(f"Number of columns available in all files: {len(list_col_in_all)}\n")

    filtered_list_col_in_all = sorted([item for item in list_col_in_all if not (item.startswith('NEE') or 
                                                                                item.startswith('GPP') or 
                                                                                item.startswith('TIMESTAMP') or
                                                                                item.endswith('_QC') or 
                                                                                item.endswith('_METHOD') or 
                                                                                item.startswith('RECO'))])

    combined_var = sorted([item for item in list_col_in_all  if item.endswith('_F')])
    era_var = sorted([item for item in list_col_in_all  if item.endswith('_ERA')])
    mds_var = sorted([item for item in list_col_in_all  if item.endswith('_MDS')])

    era_var.insert(0, 'GPP_DT_VUT_REF')
    df = df[era_var]

    correlation_matrix = df.corr()

    plt.figure(figsize=(10, 12))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation Matrix')
    plt.show()

    return df

def filter_flux_columns(data_directory, sites_table):

    list_col = []
    nan_files = []

    for index, row in sites_table.iterrows():
        print(f"\nSite ID: {row['sites_ids']}")

        # Unzip files if needed
        # file_path = os.path.join(data_directory, row['File name'])
        # with zipfile.ZipFile(file_path, 'r') as zip_ref:
        #     zip_ref.extractall(os.path.join(data_directory, row['sites_folder'])) 
        #     print(f"Unzipped: {row['sites_folder']}")
        try:
            flux_file_path = os.path.join(data_directory,row['sites_folder'],f"ICOSETC_{row['sites_ids']}_FLUXNET_DD_L2.csv" )
            flux_df = read_flux_df(flux_file_path)
            list_col.extend(flux_df.columns.values.tolist())

        except FileNotFoundError:
            print(f"File not found: {flux_file_path}")
            nan_files.append({flux_file_path}) 

    counts = Counter(list_col)
    list_col_in_all = set([item for item in list_col if counts[item] == (len(sites_table)-len(nan_files))])
    print(f"Number of columns available in all files: {len(list_col_in_all)}")

    filtered_flux_columns = sorted([item for item in list_col_in_all if not (item.startswith('NEE') or 
                                                                                item.startswith('GPP') or 
                                                                                item.startswith('TIMESTAMP') or
                                                                                item.endswith('_QC') or 
                                                                                item.endswith('_METHOD') or 
                                                                                item.startswith('RECO'))])
    
    print(f"Number of columns available in all files without carbon data: {len(filtered_flux_columns)}\n")

    insitu_cols = ['GPP_DT_VUT_REF', 'NEE_VUT_REF_QC'] + filtered_flux_columns #'GPP_DT_VUT_USTAR50'

    return insitu_cols

def get_eu_sites(df):
    # Fetch country data from the REST Countries API
    response = requests.get('https://restcountries.com/v3.1/all')
    countries = response.json()

    # Extract country codes and their regions
    european_country_codes = [country['cca2'] for country in countries if 'region' in country and country['region'] == 'Europe']
    df['IsInEurope'] = df['country'].isin(european_country_codes)
    df = df[df['IsInEurope'] == True]
    df = df.drop(columns=['IsInEurope'])
    df
    return df

def map_sites(df):
    # Convert lat/lon to geometry
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Create a Folium map centered around the average location
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=5)

    # Add points to the map
    for idx, row in gdf.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['siteType']).add_to(m)

    # Display the map
    return m

def map_sites_with_colors(df):
    
    # Convert lat/lon to geometry
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Define a color mapping based on the name
    color_mapping = {
        'Cropland': 'yellow',
        'cropland': 'yellow',
        'Forest': 'green',
        'forest': 'green',
        'Wetland': 'blue',
        'wetland': 'blue',
        'Grassland': 'magenta',
        'grassland': 'magenta',
        'Heathland': 'red',
        'Urban': 'orange',
        'urban': 'orange'
    }

    # Create a Folium map centered around the average location
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=3)

    # Add points to the map with different colors
    for idx, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4,
            popup=row['siteType'],
            color=color_mapping.get(row['siteType'], 'black'),  # Default to black if name not in mapping
            fill=True,
            fill_color=color_mapping.get(row['siteType'], 'black')
        ).add_to(m)

    # Display the map
    return m

def filter_long_ts(df, years):
    seconds_in_year = 365.25 * 24 * 60 * 60

    df['years_HH'] = df['end_HH'] - df['start_HH'] 
    df['years_HH']=df['years_HH'].dt.total_seconds() / seconds_in_year

    df = df[df['years_HH'] >= years]

    return df

def get_sites(df, years_ts, output_directory):

    df.to_csv(os.path.join(output_directory,'sites_table_complete.csv'), index=False)

    df = get_eu_sites(df)
    df = filter_long_ts(df, years_ts)
    df = df[df['ecosystemType'] != '']
    df['siteType'] = df['siteType'].str.strip()
    df['ecosystemType'] = df['ecosystemType'].str.strip()
    df['siteType'] = df['siteType'].str.lower()
    df['ecosystemType'] = df['ecosystemType'].str.lower()

    df = df[(df['siteType'] == 'forest') | (df['siteType'] == 'grassland')]

    df.to_csv(os.path.join(output_directory,'sites_table_filtered_4years.csv'), index=False) 
    print(df.shape)
    return df

def read_icos_attributes(index, sites_table, id, output_directory,axs):
    myStation = station.get(id)
    metadata = myStation.info()
    sites_table.loc[index, 'lat'] = metadata['lat']
    sites_table.loc[index, 'lon'] = metadata['lon']
    sites_table.loc[index, 'country'] = metadata['country']
    sites_table.loc[index, 'siteType'] = metadata['siteType']

    nan_ids = []
    try:
        selected_df = myStation.data(2).loc[myStation.data(2)['specLabel'] == "ETC L2 Fluxnet (half-hourly)"]
        pid = selected_df['dobj'].values.tolist()
        dobj = Dobj(pid[0])

        start, end = dobj.data['TIMESTAMP'].iloc[[0, -1]]
        sites_table.loc[index, 'start_HH'] = start
        sites_table.loc[index, 'end_HH'] = end

        # ax = dobj.data.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True)
        # ax.set_title(f'GPP_DT_VUT_REF Over Time at {id} Station')

        dobj.data.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[0])
        axs[0].set_title(f'GPP_DT_VUT_REF Over Time at {id} Station HH')

        # plt.savefig(os.path.join(output_directory, f'GPP_{id}.png'))
        # plt.close()

        sites_table.loc[index, 'ecosystemType'] = dobj.meta['specificInfo']['acquisition']['station']['specificInfo']['ecosystemType']['label']
        dobj = dobj.data
        
    except FileNotFoundError:
        print(f"Flux data not found: {id}")
        nan_ids.append({id})
        dobj = pd.DataFrame() 
    except IndexError:
        print(f"Index out of range for station {id}")
        nan_ids.append(id)
        dobj = pd.DataFrame() 
    except KeyError as e:
        print(f"KeyError: {e}. The key 'GPP_DT_VUT_REF' does not exist for station {id}.")
        nan_ids.append(id)
        dobj = pd.DataFrame() 

    return sites_table, nan_ids, axs, dobj

def process_insitu_data(data_directory, sites_folder, id, insitu_cols, sites_table, index, output_directory, axs):

    nan_files = []
    try:
        flux_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXNET_DD_L2.csv" )
        insitu = pd.read_csv(flux_file_path, index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])
        insitu = insitu[insitu_cols]

        insitu['TIMESTAMP'] = pd.to_datetime(insitu.index)
        # insitu['month'] = pd.DatetimeIndex(insitu['TIMESTAMP']).month
        # insitu['day'] = pd.DatetimeIndex(insitu['TIMESTAMP']).day

        # start, end = insitu['TIMESTAMP'].iloc[[0, -1]]
        # sites_table.loc[index, 'start_DD'] = start
        # sites_table.loc[index, 'end_DD'] = end

        # insitu.to_csv(os.path.join(output_directory, f"{id}_preprocessed_{start.strftime('%d%m%Y')}_{end.strftime('%d%m%Y')}.csv"))

        # ax = insitu.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True)
        # ax.set_title(f'GPP_DT_VUT_REF Over Time at {id} Station')

        insitu.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[1])
        axs[1].set_title(f'GPP_DT_VUT_REF Over Time at {id} Station (Daily Data)')

        # plt.savefig(os.path.join(output_directory, f'GPP_daily_{id}.png'))
        # plt.close()

        fetch_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXES_L2.csv" )
        fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)

        sites_table.loc[index, 'FETCH_MAX'] = fetch.describe()['FETCH_MAX'].loc['50%']
        sites_table.loc[index, 'FETCH_70'] = fetch.describe()['FETCH_70'].loc['50%']
        sites_table.loc[index, 'FETCH_90'] = fetch.describe()['FETCH_90'].loc['50%']

        iheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_VARINFO_FLUXNET_DD_L2.csv" )
        iheight = pd.read_csv(iheight_file_path)
        filtered_df = iheight[iheight['DATAVALUE'] == 'GPP_DT_VUT_REF']
        index_gpp = filtered_df.index.values.tolist()[0] 
        iheight = iheight[index_gpp:index_gpp+5]
        iheight_value = iheight[iheight['VARIABLE'] == 'VAR_INFO_HEIGHT']
        sites_table.loc[index, 'instrument_height'] = float(iheight_value['DATAVALUE'].values.tolist()[0])

        cheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_ANCILLARY_L2.csv" )
        cheight = pd.read_csv(cheight_file_path)
        cheight = cheight[cheight['VARIABLE_GROUP'] == 'GRP_HEIGHTC']
        cheight = cheight[cheight['VARIABLE'] == 'HEIGHTC']
        cheight['DATAVALUE'] = cheight['DATAVALUE'].astype(float)
        sites_table.loc[index, 'canopy_height'] = cheight.describe()['DATAVALUE'].loc['75%']
        sites_table['buffer'] = 100 * (sites_table['instrument_height']-sites_table['canopy_height']*2/3) #Fetch to height ratio https://www.mdpi.com/2073-4433/10/6/299
                                                                                                        #https://nicholas.duke.edu/people/faculty/katul/Matlab_footprint.html 
        sites_table['buffer_desired'] = sites_table['buffer'] / 3
        sites_table['buffer_difference'] = sites_table['buffer_desired'] - sites_table['FETCH_90']

    except FileNotFoundError:
        print(f"File not found: {flux_file_path}")
        nan_files.append({flux_file_path}) 
        insitu = pd.DataFrame() 

    except KeyError as e:
        print(f"KeyError: {e}. The key 'FETCH' does not exist for station {id}.")

    return sites_table, nan_files, axs, insitu


def plot_gpp(dobj, insitu, axs, output_directory):
    nan_plots = []
    try:
        # Plot 3: Combined Plot
        if 'GPP_DT_VUT_REF' in dobj.columns and 'GPP_DT_VUT_REF' in insitu.columns:
            # Plot 3
            dobj.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[2], label='Dobj Data')
            insitu.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[2], label='Daily Data', linestyle='--')
            axs[2].set_title(f'Combined GPP_DT_VUT_REF Over Time at {id} Station')
            axs[2].legend()

        plt.tight_layout()
        plt.savefig(os.path.join(output_directory, f'Combined_GPP_{id}.png'))
        plt.close()

    except KeyError as e:
        print(f"KeyError: {e}. The key 'GPP_DT_VUT_REF' does not exist in one of the dataframes for station {id}.")
        nan_plots.append(id)

    return axs, nan_plots

def elevation_icos_attributes(id, index, sites_table):
    myStation = station.get(id)
    metadata = myStation.info()
    sites_table.loc[index, 'eas'] = metadata['eas'] 
    return sites_table

def merge_data(data_directory, sites_folder, id, insitu_cols, directory_sentinel):
    try:

        sentinel = pd.read_csv(os.path.join(directory_sentinel,f"{id}_Vegetation_indices_processed.csv"), parse_dates=['date'])
        sentinel.rename(columns={'date':'TIMESTAMP'}, inplace=True)
        sentinel.set_index('TIMESTAMP', inplace=True)

        flux_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXNET_DD_L2.csv" )
        insitu = pd.read_csv(flux_file_path, index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])
        insitu = insitu[insitu_cols]

        merged_input = pd.merge(left= insitu, right = sentinel, how="inner", left_index = True , right_index = True)
        merged_input['TIMESTAMP'] = pd.to_datetime(merged_input.index)
        merged_input['lat'] = row['lat']
        merged_input['lon'] = row['lon']
        merged_input['elevation'] = row['eas']
        merged_input['canopy_height'] = row['canopy_height']
        merged_input['month'] = pd.DatetimeIndex(merged_input['TIMESTAMP']).month
        merged_input['day'] = pd.DatetimeIndex(merged_input['TIMESTAMP']).day
        # merged_input = merged_input[merged_input['TIMESTAMP'].dt.year >= row['years_keep']]
        start, end = merged_input['TIMESTAMP'].iloc[[0, -1]]

        merged_input.drop(columns=['TIMESTAMP'], inplace=True)

    except FileNotFoundError:
        print(f"File not found: {flux_file_path}")
        merged_input = pd.DataFrame() 
    
    return merged_input, start, end

# FUNCTIONS
def additional_gobal(df,data):
    # Adding season data
    month_to_season = {
        1: 'winter', 2: 'winter', 3: 'spring',
        4: 'spring', 5: 'spring', 6: 'summer',
        7: 'summer', 8: 'summer', 9: 'fall',
        10: 'fall', 11: 'fall', 12: 'winter'
    }
    data['season'] = data['month'].map(month_to_season)
    df_encoded = pd.get_dummies(data, columns=['season'], prefix='season')
    data[['winter', 'spring', 'summer', 'fall']] = df_encoded[['season_winter', 'season_spring', 'season_summer', 'season_fall']].astype(int)
    data.drop(columns = ['season'], inplace = True)
    
    #Adding ecosystem type data
    df = pd.get_dummies(df, columns=['ecosystemType'], prefix='biom')
    df[['biom_evergreen needleleaf forests','biom_grasslands','biom_deciduous broadleaf forests','biom_mixed forests']]=df[['biom_evergreen needleleaf forests','biom_grasslands','biom_deciduous broadleaf forests','biom_mixed forests']].astype(int)
    data['biom_evergreen needleleaf forests'] = df.loc[df['sites_ids'] == id, 'biom_evergreen needleleaf forests'].iloc[0]
    data['biom_grasslands'] = df.loc[df['sites_ids'] == id, 'biom_grasslands'].iloc[0]
    data['biom_deciduous broadleaf forests'] = df.loc[df['sites_ids'] == id, 'biom_deciduous broadleaf forests'].iloc[0]
    data['biom_mixed forests'] = df.loc[df['sites_ids'] == id, 'biom_mixed forests'].iloc[0]
    return data

In [7]:
data_directory = r'D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC'
directory_sentinel = r'D:\Proyectos2024\Agame\Output\sentinel2\VI_output'
directory_site_selection = r'D:\Proyectos2024\Agame\Output\sites_selection'
output_directory = r'D:\Proyectos2024\Agame\Output\Tables'

sites_table = pd.read_csv(os.path.join(data_directory,'!TOC.csv'))
sites_table['sites_ids'] = sites_table['File name'].apply(lambda x: x.split('_')[1])
sites_table['sites_folder'] = sites_table['File name'].apply(lambda x: x.replace('.zip', ''))
insitu_cols = filter_flux_columns(data_directory, sites_table)

sites_table = pd.read_csv(os.path.join(directory_site_selection,'sites_table_filtered_4y.csv'), sep=';')
#sites_table = sites_table[sites_table['sites_ids'] == 'IT-Tor']
#sites_table_s = sites_table[:]


Site ID: FI-Sii

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_FI-Sii_ARCHIVE_L2\ICOSETC_FI-Sii_FLUXNET_DD_L2.csv
Original number of columns: 346
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: IT-SR2

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_IT-SR2_ARCHIVE_L2\ICOSETC_IT-SR2_FLUXNET_DD_L2.csv
Original number of columns: 346
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: SE-Htm

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_SE-Htm_ARCHIVE_L2\ICOSETC_SE-Htm_FLUXNET_DD_L2.csv
Original number of columns: 348
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: GL-Dsk

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_GL-Dsk_ARCHIVE_L2\ICOSETC_GL-Dsk_FLUXNET_DD_L2.csv
Original number of columns: 344
Number of columns dropped: 14
Columns dropped: ['LE_F_MDS_QC', 'LE_CORR', 'LE_CORR_25', 'LE_CORR_75', 'LE_CORR_JOINTUNC', 'H

In [8]:
for index, row in sites_table.iterrows():
    id = row['sites_ids']
    sites_folder = row['sites_folder']
    print('Process for: ',index, id)
    merged_input, start, end = merge_data(data_directory, sites_folder, id, insitu_cols, directory_sentinel)
    merged_input = additional_gobal(sites_table, merged_input)

    # merged_input  = merged_input[merged_input['NEE_VUT_REF_QC']>0.5]
    merged_input  = merged_input.drop('NEE_VUT_REF_QC', axis=1) 
    merged_input.to_csv(os.path.join(output_directory,f"{id}_preprocessed_{start.strftime('%d%m%Y')}_{end.strftime('%d%m%Y')}.csv"))

Process for:  0 IT-SR2
Process for:  1 SE-Htm
Process for:  2 IT-Tor
Process for:  3 IT-Niv
Process for:  4 FR-Mej
Process for:  5 DE-Msr
Process for:  6 DE-Har
Process for:  7 DE-Hai
Process for:  8 CH-Dav
Process for:  9 SE-Svb
Process for:  10 SE-Nor
Process for:  11 FR-Fon
Process for:  12 FR-Bil
Process for:  13 FI-Hyy
Process for:  14 DE-Tha
Process for:  15 DE-HoH
Process for:  16 BE-Vie
Process for:  17 BE-Bra
